# 5.1 How to create deep learning neural network model

In [1]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
url = 'https://raw.githubusercontent.com/klee141/hcmld21/main/Datasets/pima-indians-diabetes.csv'

In [12]:
dataset = loadtxt(url, delimiter=',')
X = dataset[:,0:8]
y = dataset[:,8]

In [18]:
dataset.shape

(768, 9)

In [20]:
X.shape

(768, 8)

In [21]:
y.shape

(768,)

In [7]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [8]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 0s 210us/step - loss: 5.4092 - accuracy: 0.5260
Epoch 2/150
768/768 [==============================] - 0s 52us/step - loss: 2.4842 - accuracy: 0.4935
Epoch 3/150
768/768 [==============================] - 0s 50us/step - loss: 1.5427 - accuracy: 0.5378
Epoch 4/150
768/768 [==============================] - 0s 49us/step - loss: 1.1876 - accuracy: 0.5716
Epoch 5/150
768/768 [==============================] - 0s 49us/step - loss: 1.0385 - accuracy: 0.5586
Epoch 6/150
768/768 [==============================] - 0s 49us/step - loss: 0.9686 - accuracy: 0.6042
Epoch 7/150
768/768 [==============================] - 0s 50us/step - loss: 0.9002 - accuracy: 0.5990
Epoch 8/150
768/768 [==============================] - 0s 51us/step - loss: 0.8752 - accuracy: 0.6133
Epoch 9/150
768/768 [==============================] - 0s 48us/step - loss: 0.8244 - accuracy: 0.6341
Epoch 10/150
768/768 [==============================] - 0s 50us/step - loss: 0.78

768/768 [==============================] - 0s 54us/step - loss: 0.5417 - accuracy: 0.7279
Epoch 81/150
768/768 [==============================] - 0s 53us/step - loss: 0.5582 - accuracy: 0.7279
Epoch 82/150
768/768 [==============================] - 0s 50us/step - loss: 0.5469 - accuracy: 0.7461
Epoch 83/150
768/768 [==============================] - 0s 56us/step - loss: 0.5478 - accuracy: 0.7253
Epoch 84/150
768/768 [==============================] - 0s 56us/step - loss: 0.5893 - accuracy: 0.7057
Epoch 85/150
768/768 [==============================] - 0s 57us/step - loss: 0.5343 - accuracy: 0.7305
Epoch 86/150
768/768 [==============================] - 0s 55us/step - loss: 0.5511 - accuracy: 0.7214
Epoch 87/150
768/768 [==============================] - 0s 55us/step - loss: 0.5365 - accuracy: 0.7344
Epoch 88/150
768/768 [==============================] - 0s 54us/step - loss: 0.5580 - accuracy: 0.7161
Epoch 89/150
768/768 [==============================] - 0s 55us/step - loss: 0.5598 - 

In [10]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

768/768 [==============================] - 0s 22us/step
Accuracy: 72.79


In [11]:
# make class predictions with the model
predictions = model.predict_classes(X)
# summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 1 (expected 1)


# 5.2 Neural network models for multi-class classification problems

In [22]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [23]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
dataframe = pandas.read_csv(url, header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [25]:
dataframe.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [26]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [27]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [29]:
kfold = KFold(n_splits=10, shuffle=True)

In [30]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 96.67% (3.33%)
